In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Load the datasets
df_shortcuts = pd.read_csv('keystroke_data.csv')  # Adjust the path as necessary
df_eeg = pd.read_csv('eeg_recording.csv')              # Adjust the path as necessary

# Convert timestamps to datetime for easier manipulation
df_shortcuts['Timestamp'] = pd.to_datetime(df_shortcuts['Timestamp'])
df_eeg['Timestamp'] = pd.to_datetime(df_eeg['Timestamp'])

# Merge the datasets on Timestamp
threshold = pd.Timedelta(seconds=5)  # Define your time threshold here
df_eeg['target'] = 0  # Initialize the target column

# Label the target if shortcut occurs within threshold
for _, row in df_shortcuts.iterrows():
    mask = (df_eeg['Timestamp'] >= row['Timestamp']) & (df_eeg['Timestamp'] <= row['Timestamp'] + threshold)
    df_eeg.loc[mask, 'target'] = 1

# Drop rows where target is 1 to prevent overlapping windows
df_eeg = df_eeg.drop_duplicates(subset='Timestamp', keep='last')

# Features and target
X = df_eeg[['TP9', 'AF7', 'AF8', 'TP10']].values
y = df_eeg['target'].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader instances
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the TCN model
class TCN(nn.Module):
    def __init__(self, num_channels, kernel_size=2, dropout=0.2):
        super(TCN, self).__init__()
        self.tcn = nn.Sequential(
            nn.Conv1d(1, num_channels, kernel_size, padding=(kernel_size//2)),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(num_channels, 1, kernel_size, padding=(kernel_size//2)),
            nn.ReLU()
        )
        self.fc = nn.Linear(6, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension
        x = self.tcn(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return torch.sigmoid(x)

# Initialize model, loss, and optimizer
model = TCN(num_channels=10)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy}')


Epoch 1/10, Loss: 17.248435974121094
Epoch 2/10, Loss: 15.038156509399414
Epoch 3/10, Loss: 15.700034141540527
Epoch 4/10, Loss: 12.649822235107422
Epoch 5/10, Loss: 14.826929092407227
Epoch 6/10, Loss: 10.661441802978516
Epoch 7/10, Loss: 10.866759300231934
Epoch 8/10, Loss: 12.675018310546875
Epoch 9/10, Loss: 12.682861328125
Epoch 10/10, Loss: 12.947100639343262
Accuracy: 0.9166666666666666
